### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

### 1. Preprocessing the data

In [2]:
def preprocess_dataframe(df):
    """
    Preprocesses the input DataFrame with the following steps:
    1. Converts 'Deal_Date', 'maturity', 'AssumedMaturity', 'YTWDate' columns to datetime.
    2. Converts 'B_Side' column to boolean (1 for 'NATIXIS BUY', 0 for 'NATIXIS SELL').
    3. Converts 'B_Price' and 'Total_Requested_Volume' columns to integers.
    4. Fills null values in 'Tier', 'AssumedMaturity', and 'YTWDate' columns with 'UNKNOWN'.
    5. Converts 'Frequency' feature values into integers (removing 'M' from the end).
    6. Drops the unsused 'Cusip' column.

    Parameters:
    - df (DataFrame): Input DataFrame.

    Returns:
    - DataFrame: Processed DataFrame.
    """

    df = df.copy()

    # Drop null values only for columns below the threshold
    columns_to_delete_null_vales = ['MidYTM', 'Coupon', 'Ccy', 'cusip',
       'maturity', 'cdcissuerShortName', 'Frequency', 'MidPrice', 'cdcissuer',
       'company_short_name', 'BloomIndustrySubGroup', 'B_Price',
       'Total_Traded_Volume_Natixis', 'B_Side',
       'Total_Traded_Volume_Away', 'Total_Requested_Volume',
       'Total_Traded_Volume', 'Type', 'Maturity', 'ISIN', 'Deal_Date']
    df = df.dropna(subset=columns_to_delete_null_vales)

    # Convert 'B_Price', 'Total_Requested_Volume', 'Frequency' to integers
    df['Frequency'] = df['Frequency'].str.replace('M', '')
    numerical_columns = ['B_Price', 'Total_Requested_Volume', 'Frequency']
    df.dropna(subset=numerical_columns, inplace=True)
    for column in numerical_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype(int)

    # Fix the error in the B_Price column
    df = df[df['B_Price'] >= 20]

    # Replace NaT with null values in the 'Maturity' column
    df['maturity'].replace({pd.NaT: np.nan}, inplace=True)

    # Convert 'Deal_Date', 'maturity', 'AssumedMaturity', 'YTWDate' to datetime
    df['Deal_Date'] = pd.to_datetime(df['Deal_Date'])
    df['maturity'] = pd.to_datetime(df['maturity'], errors='coerce',  format='%Y-%m-%d %H:%M:%S.%f')
    df['AssumedMaturity'] = pd.to_datetime(df['AssumedMaturity'], errors='coerce')
    df['YTWDate'] = pd.to_datetime(df['YTWDate'], errors='coerce')

    # Add year, month, day for clustering 
    df['Year_dealdate'] = df['Deal_Date'].dt.year
    df['Month_dealdate'] = df['Deal_Date'].dt.month
    df['Day_dealdate'] = df['Deal_Date'].dt.day
    df['Year_maturity'] = df['maturity'].dt.year
    df['Month_maturity'] = df['maturity'].dt.month
    df['Day_maturity'] = df['maturity'].dt.day

    # Delete maturities smaller than 2021 (as deal dates starts in 2021)
    df = df[df['maturity'].dt.year >= 2021]

    # Compute number of days between maturity and deal date
    df['Days_to_Maturity'] = (df['maturity'] - df['Deal_Date']).dt.days

    # Replace null values in 'AssumedMaturity' with values from 'Maturity'
    df['AssumedMaturity'] = df['AssumedMaturity'].fillna(df['Maturity'])

    # Convert 'B_Side' column to boolean (1 for 'NATIXIS BUY', 0 for 'NATIXIS SELL')
    df = df[df['B_Side'].isin(['NATIXIS SELL', 'NATIXIS BUY'])]
    df['B_Side'] = df['B_Side'].replace({'NATIXIS BUY': 1, 'NATIXIS SELL': 0})

    # Convert null values of 'Tier'
    df['Tier'].fillna('UNKNOWN', inplace=True)

    # Lower string names 
    df['Sales_Name'] = df['Sales_Name'].str.lower()
    df['company_short_name'] = df['company_short_name'].str.lower()

    # Drop unused columns
    columns_to_drop = ['Cusip', 'Maturity']
    df.drop(columns=columns_to_drop, inplace=True)

    return df

In [21]:
# Preprocess the data
df = pd.read_csv("../data.csv")
df_preprocessed = preprocess_dataframe(df)

input_column = ['ISIN']

# Drop unnecessary columns
columns_to_drop = ['Deal_Date', 'cusip', 'Sales_Name', 'Sales_Initial', 'company_short_name', 'cdcissuer',
                   'maturity', 'AssumedMaturity']
columns_unsure = ['Instrument', 'YTWDate', 'Month_dealdate', 'Day_dealdate', 'Month_maturity', 'Day_maturity']
df_final = df_preprocessed.drop(columns_to_drop + columns_unsure, axis=1)

# Handle missing values (if any)
df_final.dropna(inplace=True)

# Standardize numerical columns
numerical_columns = ['B_Price', 'Total_Requested_Volume', 'Total_Traded_Volume_Natixis',
                     'Total_Traded_Volume_Away', 'Total_Traded_Volume', 'Coupon',
                     'MidPrice', 'MidYTM', 'SpreadvsBenchmarkMid', 'MidASWSpread',
                     'MidZSpread', 'GSpreadMid', 'MidModifiedDuration', 'MidConvexity',
                     'MidEffectiveDuration', 'MidEffectiveConvexity', 'Year_dealdate',
                     'Year_maturity', 'Days_to_Maturity']
scaler = StandardScaler()
df_final[numerical_columns] = scaler.fit_transform(df_final[numerical_columns])

# Encode categorical columns
categorical_columns = ['B_Side', 'BloomIndustrySector', 'BloomIndustryGroup', 'BloomIndustrySubGroup',
                       'cdcissuerShortName', 'Country', 'lb_Platform_2', 'Rating_Fitch',
                       'Rating_Moodys', 'Rating_SP', 'Ccy', 'Classification', 'Tier',
                       'Frequency', 'Type']
df_final = pd.get_dummies(df_final, columns=categorical_columns, drop_first=True, dtype=int)

df_final

/var/folders/qw/_kxt72614ln500hzb4s2zwt00000gn/T/ipykernel_2876/3911111742.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data.csv")


,ISIN,B_Price,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,Coupon,MidPrice,MidYTM,SpreadvsBenchmarkMid,...,Classification_Technology,Classification_Telecommunications Services,Classification_Utilities,Tier_SECDOM,Tier_SNRFOR,Tier_SUBLT2,Tier_UNKNOWN,Frequency_6,Frequency_12,Type_Stepup
12,IT0005530032,0.065155,-0.246467,-0.078542,-0.197600,-0.210594,1.549703,-0.116114,2.026412,1.431509,...,0,0,0,0,0,0,1,1,0,0
13,IT0005530032,0.065155,-0.247578,-0.078542,-0.199460,-0.212183,1.549703,-0.116114,2.026412,1.431509,...,0,0,0,0,0,0,1,1,0,0
14,IT0005530032,-0.015470,-0.246467,-0.078542,-0.197600,-0.210594,1.549703,-0.116114,2.026412,1.431509,...,0,0,0,0,0,0,1,1,0,0
15,IT0005530032,-0.015470,-0.247023,-0.078542,-0.198344,-0.211230,1.549703,-0.116114,2.026412,1.431509,...,0,0,0,0,0,0,1,1,0,0
16,IT0005530032,-0.015470,-0.245079,-0.078542,-0.195739,-0.209004,1.549703,-0.116114,2.026412,1.431509,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635408,ES0000012K20,-0.902348,-0.220087,-0.078542,-0.162254,-0.180392,-0.717426,-0.894547,-0.565436,0.104939,...,0,0,0,0,0,0,1,0,1,0
635414,ES0413860802,-0.176721,-0.136780,-0.078542,-0.050634,-0.085017,-0.082629,-0.117670,0.082053,0.100163,...,0,0,0,0,0,0,1,0,1,0
635415,IT0005439275,0.468281,-0.170103,-0.078542,-0.095281,-0.123167,-1.140623,0.445709,-0.145333,-0.974080,...,0,0,0,0,0,0,1,0,1,0
635421,XS1496770626,0.468281,-0.220087,-0.078542,-0.162254,-0.180392,-1.140623,0.502002,-0.853967,-0.163133,...,0,0,0,0,0,0,1,0,1,0
